# Quick start of the EVStabilityNet

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import math
import os


# Load model from folder

In [9]:
model = load_model('EVStabilityNet_v1', compile=False)

# Function for evaluating performance on test set

In [10]:
def misclassified_ratio(y_predict, y_train):
    """
    Arguments:
        y_predict: (ndarray Shape (m,1))   predicted y values by model
        y_train: (ndarray Shape (m,1))   actual y values from training set
    Returns: 
        ratio of misclassified elements 
        mis_idx: (ndarray Shape (n,1))  n wrong classified indices
    """
    num_examples = len(y_train)
    assert len(y_predict) == num_examples, "Error in missclassfied_ratio function"
    
    misclass = 0
    mis_idx = []
    for idx in range(num_examples):
        if y_predict[idx] != y_train[idx]:
            misclass += 1
            mis_idx.append(idx)
            
    return misclass/num_examples, mis_idx

# Function for loading test data from folders

In [30]:
def create_testset(n, folderpath = "data_testset/train_"):
    """
    Creates data used for the test set
    Arguments:
        n: number of training files specified in folderpath + str(int) (idx = 0, ..., n)
        folderpath: path and filenames for test data structed as follows:
            first line: (kappa, stability) 
            second-last line: energy profile function in 1/1000 increments on [0,1]
    Returns:
        Two tensorflow-tensors of size (n,2,640)
    """
    data1 = []
    data2 = []
    for i in range(n):
        filename = folderpath +  str(i) + ".txt"
        if os.path.isfile(filename):
            with open(filename, 'r') as f:
                
                # read the first line separately to get the first column and the second element of the first line
                first_line = f.readline().strip().split()
                kappa = float(first_line[0])
                kappa_column = np.array( [kappa for j in range(1000)], dtype=float)
                stability = float(first_line[1])
                phi_column = np.array([ float(line.strip().split()[0]) for line in f], dtype=float)
                
                # Set the energy profile function to 0 outside of the relevant values
                for idx in range(len(phi_column)):
                    if idx/999. > 1. - np.exp(-kappa):
                        phi_column[idx] = 0
                        
                data1.extend( [[kappa_column, phi_column]])
                data2.append(stability)

        else:
            print("File does not exist:", filename)

    # convert the data lists to np array
    data1_tensor = tf.convert_to_tensor(data1, dtype=tf.float32)
    data2_tensor = tf.convert_to_tensor(data2, dtype=tf.float32)
    
    # Crop the data for the networks architecture
    data1_tensor = data1_tensor[:,:,:640]

    
    return data1_tensor, data2_tensor


# Create test data, run test data through the network, calculate misclassified ratio

In [31]:
# Create test data from folder
X_test,y_test = create_testset(500, folderpath = "data_testset/test_")

# Test set evaluation
prediction_test = model.predict(X_test, verbose=0)
prob_predict_test  = tf.nn.sigmoid(prediction_test)

y_test_predict = np.array( (prob_predict_test > 0.5))

accuracy_test, mis_idx_test= misclassified_ratio(y_test_predict, y_test)
print("Missclassfied examples ratio on test set:", accuracy_test)
print("Missclassfied examples:", mis_idx_test)

Missclassfied examples ratio on test set: 0.01
Missclassfied examples: [64, 194, 242, 357, 419]
